In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

from src.generators.lcg import LCG
from src.attacks.lcg_crack import crack_lcg_params

print("=== DÉMONSTRATION : CASSAGE DE LCG ===")

# 1. Scénario : On a une "Boîte Noire" LCG avec des paramètres inconnus (ou supposés standards)
# Pour l'exemple, prenons un module m standard (2^31) mais une seed inconnue
m_known = 2**31
secret_gen = LCG(seed=123456789) # L'attaquant ne connait pas cette seed

print(f"Paramètres secrets réels : a={secret_gen.a}, c={secret_gen.c}")

# 2. L'attaquant observe quelques sorties
observed_values = [secret_gen.next_int() for _ in range(10)]
print(f"Valeurs observées : {observed_values[:5]}...")

# 3. Attaque : Reconstruction des paramètres a et c
# Il faut au moins 3 valeurs pour résoudre le système
try:
    a_found, c_found = crack_lcg_params(observed_values, m_known)
    
    print(f"\n[Attaque] Paramètres trouvés : a={a_found}, c={c_found}")
    
    # Vérification
    if a_found == secret_gen.a and c_found == secret_gen.c:
        print(">> SUCCÈS : Les paramètres ont été entièrement compromis !")
    else:
        print(">> ÉCHEC de l'attaque.")
        
    # 4. Prédiction du futur
    # On reconstruit notre propre générateur avec les paramètres volés
    # Le dernier état observé est observed_values[-1]
    last_observed = observed_values[-1]
    
    # On crée un clone
    cloned_gen = LCG(seed=last_observed, a=a_found, c=c_found, m=m_known)
    
    # On prédit la prochaine valeur
    predicted_next = cloned_gen.next_int()
    real_next = secret_gen.next_int()
    
    print(f"\n[Prédiction] Valeur prédite : {predicted_next}")
    print(f"[Réalité]    Valeur réelle  : {real_next}")
    
    assert predicted_next == real_next
    print(">> La prédiction est exacte.")

except Exception as e:
    print(f"Erreur durant l'attaque : {e}")

=== DÉMONSTRATION : CASSAGE DE LCG ===
Paramètres secrets réels : a=1103515245, c=12345
Valeurs observées : [231794730, 1126946331, 1757975480, 850994577, 1634557174]...

[Attaque] Paramètres trouvés : a=1103515245, c=12345
>> SUCCÈS : Les paramètres ont été entièrement compromis !

[Prédiction] Valeur prédite : 1434329552
[Réalité]    Valeur réelle  : 1434329552
>> La prédiction est exacte.


In [2]:
from src.generators.mt_wrapper import MT19937
from src.attacks.mt_crack import untemper, MT19937Engine

print("\n=== DÉMONSTRATION : CLONAGE D'ÉTAT MT19937 ===")

# 1. La victime : Un générateur MT19937 initialisé aléatoirement
victim_rng = MT19937(seed=54321)

# 2. Collecte de données : L'attaquant a besoin de 624 sorties consécutives de 32 bits
print("Collecte de 624 sorties...")
outputs = []
for _ in range(624):
    outputs.append(victim_rng.next_int())

print(f"Dernière sortie observée : {outputs[-1]}")

# 3. Attaque : Reconstruction de l'état interne
# L'état interne du MT correspond aux sorties APRÈS avoir inversé le "tempering"
reconstructed_state = []
for y in outputs:
    reconstructed_state.append(untemper(y))

print("État interne reconstruit avec succès.")

# 4. Clonage : On injecte cet état dans notre propre moteur MT
# Note: On utilise une classe Engine pure Python fournie dans src/attacks/mt_crack.py
# pour pouvoir injecter l'état manuellement.
clone_rng = MT19937Engine(reconstructed_state)

# 5. Vérification : Les deux générateurs doivent maintenant produire la même suite
print("\nComparaison des 10 prochaines valeurs :")
match = True
for i in range(10):
    val_real = victim_rng.next_int()
    val_clone = clone_rng.next_int()
    
    print(f"#{i+1}: Réel={val_real:<12} | Clone={val_clone:<12} | {'OK' if val_real==val_clone else 'DIFF'}")
    if val_real != val_clone:
        match = False

if match:
    print("\n>> SUCCÈS : Le générateur est cloné. Nous pouvons prédire toutes les futures clés.")
else:
    print("\n>> ÉCHEC : Le clonage n'a pas fonctionné.")


=== DÉMONSTRATION : CLONAGE D'ÉTAT MT19937 ===
Collecte de 624 sorties...
Dernière sortie observée : 1247491122
État interne reconstruit avec succès.

Comparaison des 10 prochaines valeurs :
#1: Réel=2952144577   | Clone=2952144577   | OK
#2: Réel=3553993575   | Clone=3553993575   | OK
#3: Réel=774224646    | Clone=774224646    | OK
#4: Réel=1977070210   | Clone=1977070210   | OK
#5: Réel=306646231    | Clone=306646231    | OK
#6: Réel=106539254    | Clone=106539254    | OK
#7: Réel=233184278    | Clone=233184278    | OK
#8: Réel=3398638948   | Clone=3398638948   | OK
#9: Réel=853062093    | Clone=853062093    | OK
#10: Réel=3309894807   | Clone=3309894807   | OK

>> SUCCÈS : Le générateur est cloné. Nous pouvons prédire toutes les futures clés.
